In [1]:
#input amygdala volume and mask and outputs nifti files of amygdala for each kernel value and moment
from __future__ import print_function 
import sys
import os
import logging
import six
import nrrd
import numpy as np
import SimpleITK as sitk
from radiomics import featureextractor, getFeatureClasses
import radiomics

In [2]:
#input variables
res='100' #resolution of data in microns
side='R' #side of hemisphere

#for loop that makes .nii files for every feature extracted from original volume at kernels 2,4,6,8,10
featureClasses = getFeatureClasses()

#imageName, maskName = radiomics.getTestCase('brain1')
workDir = '/host/percy/local_raid/hans/amyg/hist/'

#amygdala volume
im = sitk.ReadImage(workDir+'outputs/amyg_'+side+'_'+res+'um.nii.gz')
#mask of the amygdala
mask = sitk.ReadImage(workDir + 'outputs/amyg_'+side+'_'+res+'um_mask-bin-vF_ero5.nii.gz')

#if imageName is None or maskName is None:  # Something went wrong, in this case PyRadiomics will also log an error
#    raise Exception('Error getting testcase!')  # Raise exception to prevent cells below from running in case of "run all"

In [3]:
# Set up and run extractor
# Use hardcoded settings (separate for settings, input image types and enabled features)
settings = {}
settings['normalize'] = True
settings['normalizeScale'] = 100
settings['removeOutliers'] = 1
settings['binWidth'] = 25
settings['resampledPixelSpacing'] = None
settings['interpolator'] = 4
settings['verbose'] = True
settings['voxelBatch'] = 10
#settings['initValue'] = float('NaN')

for kernelRad in range(0,10,2):   
    settings['kernelRadius'] = kernelRad + 2
    extractor = featureextractor.RadiomicsFeatureExtractor(**settings)

    # Disable all classes
    extractor.disableAllFeatures()

    # Enable all features in firstorder
    #extractor.enableFeatureClassByName('firstorder')
    extractor.enableFeaturesByName(firstorder=['Mean', 'Variance', 'Skewness', 'Kurtosis'])

    # By default, only 'Original' (no filter applied) is enabled. Optionally enable some image types:
    extractor.disableAllImageTypes()
    extractor.enableImageTypeByName('Original')
    # extractor.enableImageTypeByName('Wavelet')
    # extractor.enableImageTypeByName('LoG', customArgs={'sigma':[4]})
    # extractor.enableImageTypeByName('Square')
    # extractor.enableImageTypeByName('SquareRoot')
    # extractor.enableImageTypeByName('Exponential')
    # extractor.enableImageTypeByName('Logarithm')
    print('Enabled input images:')
    for imageType in extractor.enabledImagetypes.keys():
        print('\t' + imageType)

    # Calculate features
    print('Calculating features')
    result = extractor.execute(im, mask, voxelBased=True)

    outPath = str('/host/percy/local_raid/hans/amyg/hist/outputs/'+res+'umfeatures_'+side+'/')
        #make proper directory is it does not exist yet:
    import os
    if not os.path.exists(outPath):
        os.mkdir(outPath)
        
    # Show output voxel based    
    for key, val in six.iteritems(result):
        if isinstance(val, sitk.Image):  # Feature map
            sitk.WriteImage(val, outPath + key + '_' + str(kernelRad + 2) + '.nrrd', True)
            print("Stored feature %s in %s" % (key, key + ".nrrd"))
        else:  # Diagnostic info
            print("\t%s: %s" %(key, val))

    # Write to nifti
    imPath = outPath
    prefix = 'original_firstorder'

    # Read images and save as nii
    #volItk = sitk.ReadImage(imPath + prefix + '_Mean_' + str(kernelRad + 2) + '.nrrd')
    #sitk.WriteImage(volItk, imPath + prefix + '_Mean_' + str(kernelRad + 2) + '_ero5_1sd.nii.gz')

    #volItk = sitk.ReadImage(imPath + prefix + '_Variance_' + str(kernelRad + 2) + '.nrrd')
    #sitk.WriteImage(volItk, imPath + prefix + '_Variance_' + str(kernelRad + 2) + '_ero5_1sd.nii.gz')

    #volItk = sitk.ReadImage(imPath + prefix + '_Skewness_' + str(kernelRad + 2) + '.nrrd')
    #sitk.WriteImage(volItk, imPath + prefix + '_Skewness_' + str(kernelRad + 2) + '_ero5_1sd.nii.gz')

    #volItk = sitk.ReadImage(imPath + prefix + '_Kurtosis_' + str(kernelRad + 2) + '.nrrd')
    #sitk.WriteImage(volItk, imPath + prefix + '_Kurtosis_' + str(kernelRad + 2) + '_ero5_1sd.nii.gz')



Enabled input images:
	Original
Calculating features
	diagnostics_Versions_PyRadiomics: v3.0.1
	diagnostics_Versions_Numpy: 1.19.1
	diagnostics_Versions_SimpleITK: 2.1.1
	diagnostics_Versions_PyWavelet: 1.1.1
	diagnostics_Versions_Python: 3.7.7
	diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 100, 'removeOutliers': 1, 'resampledPixelSpacing': None, 'interpolator': 4, 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'verbose': True, 'voxelBatch': 10, 'kernelRadius': 2, 'voxelBased': True}
	diagnostics_Configuration_EnabledImageTypes: {'Original': {}}
	diagnostics_Image-original_Hash: 7461148c6d9d40d919b3ec490371084f64db98b5
	diagnostics_Image-original_Dimensionality: 3D
	diagnostics_Image-original_Spacing: (0.10000000149011612, 0.10000000149011612, 0.10000000149011612)
	diagnostics_Image-ori

	diagnostics_Versions_PyRadiomics: v3.0.1
	diagnostics_Versions_Numpy: 1.19.1
	diagnostics_Versions_SimpleITK: 2.1.1
	diagnostics_Versions_PyWavelet: 1.1.1
	diagnostics_Versions_Python: 3.7.7
	diagnostics_Configuration_Settings: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': True, 'normalizeScale': 100, 'removeOutliers': 1, 'resampledPixelSpacing': None, 'interpolator': 4, 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True, 'binWidth': 25, 'verbose': True, 'voxelBatch': 10, 'kernelRadius': 10, 'voxelBased': True}
	diagnostics_Configuration_EnabledImageTypes: {'Original': {}}
	diagnostics_Image-original_Hash: 7461148c6d9d40d919b3ec490371084f64db98b5
	diagnostics_Image-original_Dimensionality: 3D
	diagnostics_Image-original_Spacing: (0.10000000149011612, 0.10000000149011612, 0.10000000149011612)
	diagnostics_Image-original_Size: (228, 161, 208)
	diagnostics_Image-origi

In [ ]:
import matplotlib.pyplot as plt
import nibabel as nib

In [ ]:
sig = '2'
im_mean_2 = nib.load(imPath + prefix + '_Mean' '_' + sig + '.nii.gz')
#print(im_mean_2.header)
meanData2=im_mean_2.get_fdata()
meanData2.shape
xslice = meanData2[10,:,:]
yslice = meanData2[:,10,:]
zslice = meanData2[:,:,10]
slices=[xslice,yslice,zslice]
fig, axes = plt.subplots(1, len(slices))
for i, slice in enumerate(slices):
    axes[i].imshow(slice.T, cmap="gray", origin="lower")

In [ ]:
#retrieve filters for filterbank
